
Use Pandas to do some feature engineering.

Transform categorical variables into binary variables.

Write a function to compute the number of misclassified examples in an intermediate node.

Write a function to find the best feature to split on.

Build a binary decision tree from scratch.

Make predictions using the decision tree.

Evaluate the accuracy of the decision tree.

Visualize the decision at the root node.

Important Note: In this assignment, we will focus on building decision trees where the data contain only binary (0 or 1) features. This allows us to avoid dealing with:

    Multiple intermediate nodes in a split
    
    The thresholding issues of real-valued features.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
loan_df = pd.read_csv('data/lending-club-data.csv',low_memory=False)

In [4]:
loan_df.head(1).T

,0
id,1077501
member_id,1296599
loan_amnt,5000
funded_amnt,5000
funded_amnt_inv,4975
...,...
payment_inc_ratio,8.1435
final_d,20141201T000000
last_delinq_none,1
last_record_none,1


In [5]:
#Labels  we reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan.
#safe_loans = 1 => safe
#safe_loans = -1 => risky
loan_df['safe_loans'] = loan_df['bad_loans'].apply(lambda x : +1 if x ==0 else -1)
loan_df.drop(columns=['bad_loans'])

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none,safe_loans
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,1.0,1.0,1.0,0,8.143500,20141201T000000,1,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,1.0,1.0,1.0,1,2.393200,20161201T000000,1,1,1,-1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,0,8.259550,20141201T000000,1,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,1.0,1.0,1.0,0,8.275850,20141201T000000,0,1,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,1.0,1.0,1.0,0,5.215330,20141201T000000,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122602,9856168,11708132,6000,6000,6000,60 months,23.40,170.53,E,E5,...,0.0,1.0,1.0,1,4.487630,20190101T000000,0,1,0,-1
122603,9795013,11647121,15250,15250,15250,36 months,17.57,548.05,D,D2,...,0.0,0.0,1.0,0,10.117800,20170101T000000,0,0,0,1
122604,9695736,11547808,8525,8525,8525,60 months,18.25,217.65,D,D3,...,0.0,1.0,1.0,0,6.958120,20190101T000000,0,1,0,-1
122605,9684700,11536848,22000,22000,22000,60 months,19.97,582.50,D,D5,...,1.0,0.0,1.0,0,8.961540,20190101T000000,1,0,1,-1


In [6]:
loan_df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans', 'bad_loans',
       'emp_length_num', 'grade_num', 'sub_gra



in this assignment, we will just be using 4 categorical features:

grade of the loan

the length of the loan term

the home ownership status: own, mortgage, rent

number of years of employment.

Since we are building a binary decision tree, we will have to convert these categorical features to a binary representation in a subsequent section using 1-hot encoding.


In [7]:
features = [
    'grade',
    'term',
    'home_ownership',
    'emp_length',    
]
target = 'safe_loans'
loan_df = loan_df[features + [target]]
loan_df.head()

,grade,term,home_ownership,emp_length,safe_loans
0,B,36 months,RENT,10+ years,1
1,C,60 months,RENT,< 1 year,-1
2,C,36 months,RENT,10+ years,1
3,C,36 months,RENT,10+ years,1
4,A,36 months,RENT,3 years,1


In [8]:
#can we use pandas instead?
print(sum(1 for x in loan_df[target].values if x==-1))
print(sum(1 for x in loan_df[target].values if x==1))

23150
99457



Subsample dataset to make sure classes are balanced

Just as we did in the previous assignment, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We use seed=1 so everyone gets the same results.


In [10]:
safe_loans_raw=loan_df[loan_df[target] == +1]
risky_loans_raw=loan_df[loan_df[target] == -1]

#ratio of sizes and percentage to undersample the safe loans

perc = risky_loans_raw.shape[0]/safe_loans_raw.shape[0]

risky_loans = risky_loans_raw
#below is the reason to use numpy
safe_loans = safe_loans_raw.sample(frac=perc,random_state=1)

loans_data = risky_loans._append(safe_loans)


In [12]:
print(f"{loans_data.shape[0]=}")
print(f" safe loans = {safe_loans.shape[0]/loans_data.shape[0]}")
print(f" risky loans = {risky_loans.shape[0]/loans_data.shape[0]}")

loans_data.shape[0]=46300
 safe loans = 0.5
 risky loans = 0.5



Transform categorical data into binary features

In this assignment, we will implement binary decision trees (decision trees for binary features, a specific case of categorical variables taking on two values, e.g., true/false). Since all of our features are currently categorical features, we want to turn them into binary features.

For instance, the home_ownership feature represents the home ownership status of the loanee, which is either own, mortgage or rent. For example, if a data point has the feature

{'home_ownership': 'RENT'}

we want to turn this into three features:

{
    
    'home_ownership = OWN'      : 0,
    
    'home_ownership = MORTGAGE' : 0,
    
    'home_ownership = RENT'     : 1
    
}



In [15]:
print(type(loans_data))
print(loans_data.dtypes)
categorical_variables = list(loans_data.select_dtypes(include=['object']).columns)
print(categorical_variables)

one_hot_data = pd.get_dummies(loans_data[categorical_variables], prefix=categorical_variables)
print(one_hot_data.columns)
loans_data.drop(columns=categorical_variables,axis=1,inplace=True)
loans_data = pd.concat([loans_data,one_hot_data],axis=1)
print(loans_data['grade_A'].values.sum())

<class 'pandas.core.frame.DataFrame'>
grade             object
term              object
home_ownership    object
emp_length        object
safe_loans         int64
dtype: object
['grade', 'term', 'home_ownership', 'emp_length']
Index(['grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F',
       'grade_G', 'term_ 36 months', 'term_ 60 months',
       'home_ownership_MORTGAGE', 'home_ownership_OTHER', 'home_ownership_OWN',
       'home_ownership_RENT', 'emp_length_1 year', 'emp_length_10+ years',
       'emp_length_2 years', 'emp_length_3 years', 'emp_length_4 years',
       'emp_length_5 years', 'emp_length_6 years', 'emp_length_7 years',
       'emp_length_8 years', 'emp_length_9 years', 'emp_length_< 1 year'],
      dtype='object')
6508


In [16]:
#Train Test Split
np.random.seed(1)
train_data, test_data = train_test_split(loans_data,test_size=.2)


Decision tree implementation

In this section, we will implement binary decision trees from scratch. There are several steps involved in building a decision tree. For that reason, we have split the entire assignment into several sections.
Function to count number of mistakes while predicting majority class

Recall from the lecture that prediction at an intermediate node works by predicting the majority class for all data points that belong to this node.

Now, we will write a function that calculates the number of missclassified examples when predicting the majority class. This will be used to help determine which feature is the best to split on at a given node of the tree.

Note: Keep in mind that in order to compute the number of mistakes for a majority classifier, we only need the label (y values) of the data points in the node.

** Steps to follow **:

** Step 1:** Calculate the number of safe loans and risky loans.

** Step 2:** Since we are assuming majority class prediction, all the data points that are not in the majority class are considered mistakes.

** Step 3:** Return the number of mistakes.

Now, let us write the function intermediate_node_num_mistakes which computes the number of misclassified examples of an intermediate node given the set of labels (y values) of the data points contained in the node. Fill in the places where you find 


In [17]:
def intermediate_node_num_mistakes(labels_in_node):
    #corner case: if labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    #1s in safe loan
    safe_loans_count = sum(labels_in_node==1)
    risky_loans_count = sum(labels_in_node==-1)
    
    #return number of mistakes the majority classifier makes
    return min(safe_loans_count,risky_loans_count)

In [18]:
#Test case 1
example_labels = np.array([-1,-1,1,1,1])

if intermediate_node_num_mistakes((example_labels)) == 2:
    print('Test passed')
else:
    print("Failed")

Test passed


Function to pick best feature to split on

The function best_splitting_feature takes 3 arguments:

The data (DataFrame of data which includes all of the feature columns and label column)

The features to consider for splits (a list of strings of column names to consider for splits)
    
The name of the target/label column (string)

The function will loop through the list of possible features, and consider splitting on each of them. It will calculate the classification error of each split and return the feature that had the smallest classification error when split on.

Recall that the classification error is defined as follows:

classification_error = #mistakes/#total_examples

Follow these steps:

Step 1: Loop over each feature in the feature list

Step 2: Within the loop, split the data into two groups: one group where all of the data has feature value 0 or False (we will call this the left split), and one group where all of the data has feature value 1 or True (we will call this the right split). Make sure the left split corresponds with 0 and the right split corresponds with 1 to ensure your implementation fits with our implementation of the tree building process.

Step 3: Calculate the number of misclassified examples in both groups of data and use the above formula to compute the classification error.

Step 4: If the computed error is smaller than the best error found so far, store this feature and its error.

This may seem like a lot, but we have provided pseudocode in the comments in order to help you implement the function correctly.

Note: Remember that since we are only dealing with binary features, we do not have to consider thresholds for real-valued features. This makes the implementation of this function much easier.


In [41]:
def best_splitting_feature(data,features,target):
    
    best_feature = None 
    best_error = 10
    #since error is always <=1, we should initialize it with something larger than 1
    
    for feature in features:
        #left split
        left_split = data[data[feature] == 0]
        #right split
        right_split = data[data[feature] ==1]
        
        left_mistakes = intermediate_node_num_mistakes(left_split[target])
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
        print(f"{feature=},{left_mistakes=},{right_mistakes=}")
        
        error = (left_mistakes + right_mistakes)/data.shape[0]
        
        if error < best_error:
            best_error = error
            best_feature =feature
    return best_feature

In [40]:
print(loans_data[loans_data['term_ 36 months'] == True].shape[0],
loans_data[loans_data['term_ 36 months'] == False].shape[0]
      )

34629 11671


In [42]:
features = list(loans_data.columns)
features.remove('safe_loans')

if best_splitting_feature(train_data,features,'safe_loans') == 'term_ 36 months':
    print("pass")
else:
    print("Fail")

feature='grade_A',left_mistakes=14560,right_mistakes=1243
feature='grade_B',left_mistakes=12725,right_mistakes=4345
feature='grade_C',left_mistakes=13590,right_mistakes=4454
feature='grade_D',left_mistakes=14457,right_mistakes=2640
feature='grade_E',left_mistakes=16246,right_mistakes=1115
feature='grade_F',left_mistakes=17282,right_mistakes=455
feature='grade_G',left_mistakes=18201,right_mistakes=128
feature='term_ 36 months',left_mistakes=3308,right_mistakes=12436
feature='term_ 60 months',left_mistakes=12436,right_mistakes=3308
feature='home_ownership_MORTGAGE',left_mistakes=9423,right_mistakes=8030
feature='home_ownership_OTHER',left_mistakes=18470,right_mistakes=23
feature='home_ownership_OWN',left_mistakes=16923,right_mistakes=1532
feature='home_ownership_RENT',left_mistakes=9598,right_mistakes=7830
feature='emp_length_1 year',left_mistakes=17171,right_mistakes=1295
feature='emp_length_10+ years',left_mistakes=13341,right_mistakes=5064
feature='emp_length_2 years',left_mistakes=16


Building the tree

With the above functions implemented correctly, we are now ready to build our decision tree. Each node in the decision tree is represented as a dictionary which contains the following keys and possible values:

{
    'is_leaf'            : True/False.
    
    'prediction'         : Prediction at the leaf node.
    
    'left'               : (dictionary corresponding to the left tree).
    
    'right'              : (dictionary corresponding to the right tree).
    
    'splitting_feature'  : The feature that this node splits on.
    
}

First, we will write a function that creates a leaf node given a set of target values. Fill in the places where you find ## YOUR CODE HERE. There are three places in this function for you to fill in.


In [49]:
def create_leaf(target_values):
    leaf = {
        'splitting_feature' :None,
        'left' :None,
        'right':None,
        'is_leaf' : True
    }
    num_ones = target_values[target_values==+1].shape[0]
    num_minus_ones = target_values[target_values==-1].shape[0]
    
    #for the leaf node, set the prediction to be the majority class.
    leaf['prediction'] = -1 #default
    if num_ones > num_minus_ones:
        leaf['prediction'] = 1
    
    return leaf

We have provided a function that learns the decision tree recursively and implements 3 stopping conditions:

Stopping condition 1: All data points in a node are from the same class.

Stopping condition 2: No more features to split on.
    
Additional stopping condition: In addition to the above two stopping conditions covered in lecture, in this assignment we will also consider a stopping condition based on the max_depth of the tree. By not letting the tree grow too deep, we will save computational effort in the learning process.


In [44]:
def decision_tree_create(data,features,target,current_depth=0,max_depth=10):
    remaining_features = features[:]
    target_values = data[target]
    print(current_depth,target_values.shape[0])
    
    if intermediate_node_num_mistakes(target_values) ==0:
        print("Stopping condition reached")
        return create_leaf(target_values)
    
    if len(remaining_features) ==0:
        print("Stopping 2")
        
    if current_depth >= max_depth:
        print("max depth reached, stopping now")
        return create_leaf(target_values)
    
    splitting_feature = best_splitting_feature(data,remaining_features,target)
    
    left_split = data[data[splitting_feature] ==0]
    right_split = data[data[splitting_feature] ==1]
    remaining_features.remove(splitting_feature)
    print(splitting_feature,len(left_split), len(right_split))

    if left_split.shape[0] == data.shape[0]:
        return create_leaf(left_split[target])

        if right_split.shape[0] == data.shape[0]:
            return create_leaf(right_split[target])
        

    left_tree = decision_tree_create(left_split,remaining_features,target, current_depth+1, max_depth)        
    right_tree = decision_tree_create(right_split,remaining_features,target, current_depth+1, max_depth)    
    
    return  {
        'splitting_feature' :splitting_feature,
        'left' :left_tree,
        'right':right_tree,
        'predition' :None,
        'is_leaf' : False
    }

In [45]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1+count_nodes(tree['left']) + count_nodes(tree['right'])

In [50]:
small_data_decision_tree = decision_tree_create(train_data,features, 'safe_loans',max_depth=3)
if count_nodes(small_data_decision_tree) ==13:
    print("Pass")
else:
    print("Failed")

0 37040
feature='grade_A',left_mistakes=14560,right_mistakes=1243
feature='grade_B',left_mistakes=12725,right_mistakes=4345
feature='grade_C',left_mistakes=13590,right_mistakes=4454
feature='grade_D',left_mistakes=14457,right_mistakes=2640
feature='grade_E',left_mistakes=16246,right_mistakes=1115
feature='grade_F',left_mistakes=17282,right_mistakes=455
feature='grade_G',left_mistakes=18201,right_mistakes=128
feature='term_ 36 months',left_mistakes=3308,right_mistakes=12436
feature='term_ 60 months',left_mistakes=12436,right_mistakes=3308
feature='home_ownership_MORTGAGE',left_mistakes=9423,right_mistakes=8030
feature='home_ownership_OTHER',left_mistakes=18470,right_mistakes=23
feature='home_ownership_OWN',left_mistakes=16923,right_mistakes=1532
feature='home_ownership_RENT',left_mistakes=9598,right_mistakes=7830
feature='emp_length_1 year',left_mistakes=17171,right_mistakes=1295
feature='emp_length_10+ years',left_mistakes=13341,right_mistakes=5064
feature='emp_length_2 years',left_mis

In [51]:
my_decision_tree = decision_tree_create(train_data,features,'safe_loans',max_depth=6)

0 37040
feature='grade_A',left_mistakes=14560,right_mistakes=1243
feature='grade_B',left_mistakes=12725,right_mistakes=4345
feature='grade_C',left_mistakes=13590,right_mistakes=4454
feature='grade_D',left_mistakes=14457,right_mistakes=2640
feature='grade_E',left_mistakes=16246,right_mistakes=1115
feature='grade_F',left_mistakes=17282,right_mistakes=455
feature='grade_G',left_mistakes=18201,right_mistakes=128
feature='term_ 36 months',left_mistakes=3308,right_mistakes=12436
feature='term_ 60 months',left_mistakes=12436,right_mistakes=3308
feature='home_ownership_MORTGAGE',left_mistakes=9423,right_mistakes=8030
feature='home_ownership_OTHER',left_mistakes=18470,right_mistakes=23
feature='home_ownership_OWN',left_mistakes=16923,right_mistakes=1532
feature='home_ownership_RENT',left_mistakes=9598,right_mistakes=7830
feature='emp_length_1 year',left_mistakes=17171,right_mistakes=1295
feature='emp_length_10+ years',left_mistakes=13341,right_mistakes=5064
feature='emp_length_2 years',left_mis

In [52]:


def classify(tree, x, annotate = False):
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
            print ("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
            print ("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [53]:
test_data.iloc[0]

safe_loans                    -1
grade_A                    False
grade_B                    False
grade_C                    False
grade_D                    False
grade_E                     True
grade_F                    False
grade_G                    False
term_ 36 months             True
term_ 60 months            False
home_ownership_MORTGAGE    False
home_ownership_OTHER       False
home_ownership_OWN         False
home_ownership_RENT         True
emp_length_1 year          False
emp_length_10+ years        True
emp_length_2 years         False
emp_length_3 years         False
emp_length_4 years         False
emp_length_5 years         False
emp_length_6 years         False
emp_length_7 years         False
emp_length_8 years         False
emp_length_9 years         False
emp_length_< 1 year        False
Name: 121355, dtype: object

In [54]:
print(f'Predicted class {classify(my_decision_tree,test_data.iloc[0])}')

Predicted class -1


In [55]:
classify(my_decision_tree, test_data.iloc[0], annotate=True)

Split on term_ 36 months = True
Split on grade_D = False
Split on grade_E = True
At leaf, predicting -1


-1